In [ ]:
from spottunet.dataset.cc359 import *
from spottunet.split import one2one
from models.spottune_unet import UNet2D
from models.resnet import resnet
from spottunet.utils import sdice
from dpipe.im.metrics import dice_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler 

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from PIL import Image

from monai import transforms as T
from monai.transforms import Compose, apply_transform
from fastprogress.fastprogress import master_bar, progress_bar


import json
import nibabel as nib
import pandas as pd
import numpy as np
from scipy import ndimage
from dpipe.im.shape_ops import zoom
import cv2
import os
import gc
from collections import defaultdict
from pathlib import Path
import segmentation_models_pytorch as smp

import matplotlib.pyplot as plt

DEBUG=False

torch.cuda.set_device('cuda:0')

In [ ]:
import wandb
from configs.config_spottune import CFG
from utils import *

def class2dict(f):
    return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

from dataset.dataloader import *
from dataset.loader import *

In [ ]:
from trainer_spottune import SpotTuneTrainer
from dataset.dataloader import *
from dataset.loader import *
from dataset.dataloader_utils import *
from dataset.augment import *

In [ ]:
if DEBUG:
    cc359_df = pd.read_csv(f"{CFG.dataset_path}/meta.csv",delimiter=",")
    seed = 0xBadCafe
    pretrained = True
    n_first_exclude = 5
    n_exps = 30
    split = one2one(cc359_df, val_size=CFG.val_size, n_add_ids=CFG.n_add_ids,
                train_on_add_only=pretrained, seed=seed)[n_first_exclude:n_exps]
    train_df = cc359_df.iloc[split[0][0]].reset_index()

    sa_x,sa_y = create_shared_arrays(CFG,train_df,root_dir=CFG.dataset_path)
    train_dataset = CC359_Dataset(CFG,df=train_df,root_dir=CFG.dataset_path,
                                  voxel_spacing=CFG.voxel_spacing,transforms=tfms,
                                  mode="train", cache=True, cached_x=sa_x, cached_y=sa_y)
    for x,y in train_dataset:
        plt.imshow(x.squeeze(), "gray")
        plt.show()
        plt.imshow(y)
        plt.show()

In [ ]:
setup2k_reg_opt = {3: {1: 0.000, }, 2: {1: 0.005, 3: 0.007, 6: 0.005, 12: 0.005, 24: 0.012, 36: 0.007, 48: 0.010 },
                   1: {1: 0.005, 3: 0.007, 6: 0.005, 12: 0.005, 24: 0.012, 36: 0.007, 48: 0.010}}

def run_fold(fold):
    result_dir = CFG.results_dir + "/mode_"+str(fold)
    os.makedirs(result_dir, exist_ok=True)
    #wandb.tensorboard.patch(root_logdir=result_dir+"/logs")
    run = wandb.init(project="domain_shift",
                     group=CFG.model_name,
                     name=f"mode_{str(fold)}",
                     job_type="spottune_gamma",
                     config=class2dict(CFG),
                     reinit=True,
                     sync_tensorboard=True)
    
    writer = SummaryWriter(log_dir=result_dir+"/logs")
    cc359_df = pd.read_csv(f"{CFG.dataset_path}/meta.csv",delimiter=",")
    
    k_reg=setup2k_reg_opt[CFG.n_add_ids][CFG.slice_sampling_interval]
    seed = 0xBadCafe
    pretrained = True
    n_first_exclude = 5
    n_exps = 30
    split = one2one(cc359_df, val_size=CFG.val_size, n_add_ids=CFG.n_add_ids,
                train_on_add_only=pretrained, seed=seed)[n_first_exclude:n_exps]
        
    train_df = cc359_df.iloc[split[fold][0]].reset_index()
    valid_df = cc359_df.iloc[split[fold][1]].reset_index()
    test_df  = cc359_df.iloc[split[fold][2]].reset_index()

    print("Caching Train Data ...")
    
    sa_x,sa_y = create_shared_arrays(CFG,train_df,root_dir=CFG.dataset_path)
    train_dataset = CC359_Dataset(CFG,df=train_df,root_dir=CFG.dataset_path,
                                  voxel_spacing=CFG.voxel_spacing,transforms=get_transforms(CFG.tfms),
                                  mode="train", cache=True, cached_x=sa_x, cached_y=sa_y)
    
    valid_dataset = CC359_Dataset(CFG,df=valid_df,root_dir=CFG.dataset_path,
                                  voxel_spacing=CFG.voxel_spacing,
                                  transforms=get_transforms("default"),mode="val", cache=False)
    test_dataset = CC359_Dataset(CFG,df=test_df,root_dir=CFG.dataset_path,
                                  voxel_spacing=CFG.voxel_spacing,
                                  transforms=get_transforms("default"),mode="test", cache=False)
    
    train_loader = DataLoader(train_dataset,
                                              batch_size=CFG.bs,
                                              shuffle=True,
                                              num_workers=CFG.num_workers,
                                              sampler=None,
                                              collate_fn=fast_collate,
                                              pin_memory=False)
    valid_loader = DataLoader(valid_dataset, 
                              batch_size=1,shuffle=False,
                              num_workers=1,pin_memory=True)
    test_dataloader = DataLoader(test_dataset, 
                                  batch_size=1,shuffle=False,
                                  num_workers=1,pin_memory=False)
    
    model = UNet2D(n_chans_in=CFG.n_chans_in, n_chans_out=CFG.n_chans_out, n_filters_init=CFG.n_filters)
    
    model_policy = resnet(num_class=64)
    
    load_model_state_fold_wise(architecture=model, baseline_exp_path=CFG.baseline_exp_path, exp=fold,
                               modify_state_fn=modify_state_fn_spottune, n_folds=len(cc359_df.fold.unique()),
                               n_first_exclude=n_first_exclude),
    freeze_model_spottune(model)
    
    model.to(CFG.device)
    model_policy.to(CFG.device)
    
    # Optims
    optim_dict = dict(optim=CFG.optim,lr=CFG.lr,weight_decay=CFG.wd)
    optimizer_main = get_optimizer(model, **optim_dict)
    optimizer_policy = get_optimizer(model_policy, **optim_dict)
        # Scheduler
    if CFG.scheduler==torch.optim.lr_scheduler.OneCycleLR:
        steps = max(len(train_loader),1)
        scheduler_main = CFG.scheduler(optimizer_main, max_lr=CFG.lr, 
                                       steps_per_epoch=steps,
                                       epochs=CFG.epochs)
        scheduler_policy = CFG.scheduler(optimizer_policy, max_lr=CFG.lr, 
                                       steps_per_epoch=steps, 
                                       epochs=CFG.epochs)
    elif CFG.scheduler==torch.optim.lr_scheduler.MultiplicativeLR:
        scheduler_main = CFG.scheduler(optimizer_main, lr_lambda=lambda epoch: CFG.scheduler_multi_lr_fact )
        scheduler_policy = CFG.scheduler(optimizer_policy, lr_lambda=lambda epoch: CFG.scheduler_multi_lr_fact )
    
    criterion = CFG.crit



    
    trainer = SpotTuneTrainer(CFG,
                      model,
                      model_policy,
                      CFG.device, 
                      optimizer_main,
                      scheduler_main,
                      optimizer_policy,
                      scheduler_policy,
                      criterion,writer,fold,
                      CFG.max_norm,
                      CFG.temperature,k_reg,CFG.reg_mode)
    
    history = trainer.fit(
            CFG.epochs, 
            train_loader, 
            valid_loader, 
            f"{result_dir}/", 
            CFG.epochs,
        )
    
    
    del train_loader
    del valid_loader
    del train_dataset
    del valid_dataset
    gc.collect()
    torch.cuda.empty_cache()
    
    trainer.test(test_dataloader,result_dir)
    td_sdice = get_target_domain_metrics(CFG.dataset_path,Path(CFG.results_dir),fold)
    #wandb.log(td_sdice)
    writer.close()
    #run.finish()
    del trainer
    gc.collect()
    

In [ ]:
for fold in CFG.fold:
    run_fold(fold)

In [ ]:
def get_random_slice(arrays, interval: int = 1):
    slc = np.random.randint(arrays[0].shape[-1] // interval) * interval
    return tuple(array[..., slc] for array in arrays)

array = np.zeros((5,256,256,256))
interval = 48
arrays = get_random_slice(array,interval)
slc = np.random.randint(arrays[0].shape[-1] // interval) * interval
slc

In [ ]:
0 48 96 144 192

In [ ]:
192+48

In [ ]:
def test_run(fold):
    result_dir = CFG.results_dir + "/mode_"+str(fold)
    os.makedirs(result_dir, exist_ok=True)
    
    cc359_df = pd.read_csv(f"{CFG.dataset_path}/meta.csv",delimiter=",")

    model = UNet2D(n_chans_in=CFG.n_chans_in, n_chans_out=CFG.n_chans_out, n_filters_init=CFG.n_filters)
    model.load_state_dict(torch.load(f"{result_dir}/mode_{fold}_best_epoch_model.pth")["model_state_dict"])    
    freeze_model_spottune(model)
    model.to(CFG.device)
    
    model_policy = resnet(num_class=64)
    model_policy.to(CFG.device)
    model_policy.load_state_dict(torch.load(f"{result_dir}/mode_{fold}_best_epoch_model.pth")["model_policy_state_dict"])  


    seed = 0xBadCafe
    pretrained = True
    n_first_exclude = 5
    n_exps = 30
    split = one2one(cc359_df, val_size=CFG.val_size, n_add_ids=CFG.n_add_ids,
                train_on_add_only=pretrained, seed=seed)[n_first_exclude:n_exps]

    test_df  = cc359_df.iloc[split[fold][2]].reset_index()

    test_dataset = CC359_Dataset(CFG,df=test_df,root_dir=CFG.dataset_path,
                                  voxel_spacing=CFG.voxel_spacing,
                                  transforms=None,mode="test", cache=False)
    trainer = SpotTuneTrainer(CFG,
                              model,
                              model_policy,
                              CFG.device, 
                              None,
                              None,
                              None,
                              None,
                              None,
                              None,fold,
                              CFG.max_norm,
                              CFG.temperature,CFG.k_reg,CFG.reg_mode)

    test_dataloader = DataLoader(test_dataset, 
                                  batch_size=1,shuffle=False,
                                  num_workers=1,pin_memory = False)

    trainer.test(test_dataloader, result_dir)
    td_sdice = get_target_domain_metrics(CFG.dataset_path,Path(CFG.results_dir),fold)
    print(td_sdice)
    return td_sdice

#test_run(0)

In [ ]:
cc359_df = pd.read_csv(f"{CFG.dataset_path}/meta.csv",delimiter=",")
seed = 0xBadCafe
pretrained = True
n_first_exclude = 5
n_exps = 30
split = one2one(cc359_df, val_size=CFG.val_size, n_add_ids=CFG.n_add_ids,
            train_on_add_only=pretrained, seed=seed)[n_first_exclude:n_exps]


In [ ]:

for i in np.arange(0,25):
    model = cc359_df.iloc[split[i][0]]['tomograph_model'].unique()
    tsla = cc359_df.iloc[split[i][0]]['tesla_value'].unique()
    print(str(model),str(tsla))

In [ ]:

    Siemens15
Siemens3
    Ge15
    ge3
    philips15
    philips3

    siemens15
    siemens3
ge15
    ge3
    philips15
    philips3


In [ ]:
import json
baseline = "baseline_results/baseline23_normed"
mode_domain = ["Siemens15", "Ge15", "Ge3", "philips15", "philips3"]
#paths = ["exp_03_no_aug", "exp_03_rrc", "exp_03_gaussianblur", "exp_03_ssi_48_nid_1_gamma"]
paths = ["exp_ssi_48_nid_1_norm"]
results = []
for path in paths:
    mode_dict = {}
    with open(f"spottune_results/{path}/mode_{str(3)}/sdice_score.json","r") as f:
            data = json.load(f)
            mode_dict[mode_domain[mode]] = np.mean(list(data.values()))
    modes = [3]
    for mode in modes:
        
        with open(f"spottune_results/{path}/mode_{str(mode)}/sdice_score.json","r") as f:
            data = json.load(f)
            mode_dict[mode_domain[mode]] = np.mean(list(data.values()))
    mode_dict[' '] = path
    results.append(mode_dict)


In [ ]:
df = pd.DataFrame.from_records(results, index=' ')
df

In [ ]:
np.mean(df.iloc[0:-1] - df.iloc[-1], axis=1)